### Cleaning dataset from uknown locations

In [14]:
import pandas as pd

# Charger le dataset
file_path = "meteorite_landings.csv"  # Remplacez par le chemin vers votre fichier
data = pd.read_csv(file_path)

# Supprimer les lignes où reclat ou reclong sont null ou égales à 0.000000
data_cleaned = data.dropna(subset=['reclat', 'reclong'])  # Supprime les valeurs nulles
data_cleaned = data_cleaned[(data_cleaned['reclat'] != 0.0) & (data_cleaned['reclong'] != 0.0)]

# Ajouter des colonnes "Country" et "Continent" avec des valeurs par défaut uniquement si elles sont manquantes
if 'Country' not in data_cleaned.columns:
    data_cleaned['Country'] = None
if 'Continent' not in data_cleaned.columns:
    data_cleaned['Continent'] = None

# Remplir les colonnes vides avec les valeurs par défaut
data_cleaned['Country'].fillna('France', inplace=True)
data_cleaned['Continent'].fillna('Europe', inplace=True)

# Sauvegarder le dataset nettoyé et enrichi
output_path = "meteoriteLandings.csv"
data_cleaned.to_csv(output_path, index=False)

print(f"Dataset nettoyé, enrichi et sauvegardé sous {output_path}")


Dataset nettoyé, enrichi et sauvegardé sous meteoriteLandings.csv


In [15]:
import pandas as pd

# Charger le dataset enrichi
file_path = "meteoriteLandings.csv"
data = pd.read_csv(file_path)

# Construire le dictionnaire des continents et pays
continent_country_dict = (
    data.groupby('Continent')['Country']
    .apply(lambda x: sorted(x.dropna().unique()))
    .to_dict()
)

# Afficher le dictionnaire
for continent, countries in continent_country_dict.items():
    print(f"{continent}: {countries}")


Africa: ['Algeria', 'Angola', 'Burkina Faso', 'Cameroon', 'Central African Republic', 'Chad', 'DR Congo', 'Egypt', 'Eswatini', 'Ethiopia', 'Ghana', 'Kenya', 'Lesotho', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Morocco', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Tunisia', 'Uganda', 'Western Sahara', 'Zambia', 'Zimbabwe']
Americas: ['Argentina', 'Brazil', 'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Ecuador', 'Mexico', 'Peru', 'United States', 'Uruguay', 'Venezuela']
Antarctic: ['Antarctica']
Asia: ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bangladesh', 'Cambodia', 'China', 'India', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Laos', 'Lebanon', 'Mongolia', 'Myanmar', 'Oman', 'Pakistan', 'Philippines', 'Saudi Arabia', 'South Korea', 'Syria', 'Thailand', 'Turkmenistan', 'Türkiye', 'Uzbekistan', 'Vietnam', 'Yemen']
Europe: ['Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia', 'Czechia', 'Denmark', 'Esto

### Adding country and continent for 2500 rows (API limitation)

In [10]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import time

# Clé de l'API OpenCage
key = '17fa3e0b85b94642b181fec6406ead7b'  # Remplacez par votre clé
geocoder = OpenCageGeocode(key)

# Charger le dataset nettoyé
file_path = "meteorites_landing.csv"  # Remplacez par votre fichier nettoyé
data = pd.read_csv(file_path)

# Ajouter les colonnes vides pour country et continent
data['country'] = None
data['continent'] = None

# Fonction pour obtenir les informations de localisation
def get_location_info(lat, lng):
    try:
        results = geocoder.reverse_geocode(lat, lng, no_annotations=True, language='en')
        if results:
            components = results[0]['components']
            country = components.get('country', 'France')
            continent = components.get('continent', 'Europe')
            return country, continent
    except Exception as e:
        print(f"Erreur pour lat: {lat}, lng: {lng} - {e}")
    return 'France', 'Europe'

# Appliquer pour les 2500 premières lignes
for idx in range(min(len(data), 2500)):
    lat = data.at[idx, 'reclat']
    lng = data.at[idx, 'reclong']
    if pd.notna(lat) and pd.notna(lng):
        country, continent = get_location_info(lat, lng)
        data.at[idx, 'country'] = country
        data.at[idx, 'continent'] = continent
    time.sleep(1)  # Respecter les limitations de l'API (1 requête par seconde)

# Ajouter des valeurs par défaut pour les autres lignes
data.loc[2500:, 'country'] = 'France'
data.loc[2500:, 'continent'] = 'Europe'

# Sauvegarder le dataset avec les nouvelles colonnes
output_path = "dataset_with_country_continent.csv"
data.to_csv(output_path, index=False)

print(f"Dataset mis à jour et sauvegardé sous {output_path}")


KeyboardInterrupt: 

In [16]:
import pandas as pd

# Charger le fichier CSV
data = pd.read_csv("meteoriteLandings.csv")

# Calculer la moyenne de la colonne 'mass' en excluant les NaN
mean_mass = data['mass'].mean()

# Remplacer les NaN dans la colonne 'mass' par la moyenne
data['mass'].fillna(mean_mass, inplace=True)

# Enregistrer les modifications dans le fichier CSV
data.to_csv("meteoriteLandings.csv", index=False)
